In [15]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import math

#FIXCONFIG: Add the path to a config file
# global_path = "/root/UR5-Pick-and-Place-Simulation/ml"
global_path = "."
data_path = global_path + "/dev/example_data/"

def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [4]:
# Parameters
inf = 1e8
W,L,H = 10,10,3
x_range = [-0.75, 0.83]
y_range = [0.02, 0.86]
z_range = [0.10, 0.53]

In [16]:
# Methods for extraction
def get_vel_index(record):
    # returns vi form (0 to V-1)
    thresholds = [0.33,0.67,inf] #FIXME: Compute the actual velocity ranges
    v = record[3]
    for i,t in enumerate(thresholds):
        if v < t:
            return i

def get_pos_index(record):
    global W,L,H
    x,y,z = record[:3]
    w_pct = (x-x_range[0])/(x_range[1] - x_range[0])
    wi = math.floor(w_pct * W)
    l_pct = (y-y_range[0])/(y_range[1] - y_range[0])
    li = math.floor(l_pct * L)
    h_pct = (z-z_range[0])/(z_range[1] - z_range[0])
    hi = math.floor(h_pct * H)
    return wi,li,hi
    

def get_time_elapsed(record):
    return record['time_elapsed']


In [4]:
#Methods for exploration


In [17]:
te_data = []
lf = os.listdir(data_path)
records = []
for f in lf:
    d = read_json_file(data_path + f)
    records.append(d)

In [7]:
np.array(records[0]['data']).shape

(225, 18)

In [8]:
records

[{'time_elapsed': 28.138182878494263,
  'data': [[0.11103303620663105,
    0.4734454381135284,
    0.35213991795983113,
    2.375392119002124e-06,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0.11103303574969695,
    0.473445421144194,
    0.35214005821725636,
    1.4128096980904312e-06,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0.11103303590621866,
    0.4734453959279835,
    0.3521402948602519,
    2.3798274959763825e-06,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0.11091909651325133,
    0.4724231977279683,
    0.35030199682884966,
    0.021064688456375033,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0.11095812592415244,
    0.4676946068290422,
    0.3535834216003093,
    0.05755766127551688,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
 

In [24]:
def get_ranges(records):
    
    x_r, y_r, z_r, v_r = [inf,-inf],[inf,-inf],[inf,-inf],[inf,-inf]
    for r in records:
        d = np.array(r['data'])
        x_r = [min(x_r[0], np.min(d[:,0])),max(x_r[1], np.max(d[:,0]))]
        y_r = [min(y_r[0], np.min(d[:,1])),max(y_r[1], np.max(d[:,1]))]        
        z_r = [min(z_r[0], np.min(d[:,2])),max(z_r[1], np.max(d[:,2]))]        
        v_r = [min(v_r[0], np.min(d[:,3])),max(v_r[1], np.max(d[:,3]))]   
    return {
        'x': x_r,
        'y': y_r,
        'z': z_r,
        'v': v_r
    }

In [25]:
print(get_ranges(records))

{'x': [-0.5929082601540181, 0.6853267354576419], 'y': [0.17368468718903904, 0.7166968115339235], 'z': [0.10820680908581316, 0.5326495547100967], 'v': [1.1526994038027358e-07, 0.9997631797204787]}


In [55]:
def run_evol_matrix(records):
    # Record evolution metrics and return the final matrix
    M = np.zeros((W+2,L+2,H+2),dtype='int8')
    num_covered_cells = []
    te_data = []
    cov_cells = 0
    for record in records:
        for r in record['data']:
            v = get_vel_index(r)
            pos = get_pos_index(r)
#             print(f"The record {r[:4]} got the cell {pos} | vel {v}")
#             print(f"Vel {v} | Marking with {1<<v}")
            M[pos] |= 1<<(v)
            # The velocities are stored in a binary signature: 100 => only the second highest (fastest) velocity range was covered
            # Potential metric: how many cells have non zero velocity: sum(M>0)
            cov_cells = np.sum(M>0)
        num_covered_cells.append(cov_cells)
        te_data.append(record['time_elapsed'])
        # TODO: Add more Metrics to be plot

    d = {
        "matrix": M,
        "Covered Cells": num_covered_cells,
        "Time Elapsed": te_data
    }
    return d

def generate_plots(D):
    # plot time elapsed
    te_data = D["Covered Cells"]
    print(D)
    plt.scatter(range(len(te_data)), te_data)
    # plt.xticks([2,4,6,8,10,12,14,16,18,20])
    plt.xlabel("Test Case #")
    plt.ylabel("Time Elapsed (seconds)")
    plt.grid()
    plt.show()

dres = run_evol_matrix(records)

The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 2
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of

The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 1
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of M is 3
The new value of

In [57]:
dres['Covered Cells']

[16, 42, 54, 61, 61, 72, 76, 78, 82, 89, 91, 99, 106, 107, 107, 107, 108, 113]

In [32]:
108 / (10*10*3)

0.36

In [40]:
V = np.array(records[0]['data'])
V[:,3]

array([2.37539212e-06, 1.41280970e-06, 2.37982750e-06, 2.10646885e-02,
       5.75576613e-02, 1.08003475e-01, 1.57157065e-01, 1.92313204e-01,
       1.59819401e-01, 2.22836897e-01, 2.06356367e-01, 3.07065485e-01,
       2.18520659e-01, 1.89704425e-01, 2.36334038e-01, 3.53237645e-01,
       2.10552324e-01, 1.23869505e-01, 1.58944780e-01, 1.95146995e-01,
       1.04175472e-01, 4.75631032e-02, 1.53983644e-02, 8.41977115e-05,
       2.12503504e-05, 3.18124069e-06, 1.31209175e-06, 5.56288314e-07,
       1.49967579e-04, 1.50627014e-04, 1.60856071e-05, 8.14074279e-07,
       4.72460830e-07, 5.72128604e-07, 5.66061674e-07, 5.60945839e-07,
       5.61297779e-07, 7.01666863e-07, 5.61262625e-07, 4.20910064e-07,
       8.41729322e-07, 4.20819160e-07, 6.66791039e-03, 1.86507178e-02,
       5.05082789e-02, 3.86813486e-02, 8.00361242e-02, 9.87383468e-02,
       5.87748509e-02, 9.67920531e-02, 6.43333389e-02, 5.47525558e-02,
       4.98063466e-02, 1.87054993e-02, 2.18089228e-02, 6.27186315e-03,
      

[0.11091909651325133,
 0.4724231977279683,
 0.35030199682884966,
 0.021064688456375033,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]